<a href="https://colab.research.google.com/github/nikinuk/fact_checking_com_GEMINI/blob/main/quick_fact_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Preparação do ambiente de execução. Única instalação necessária, não padrão do Colab é a própria SDK do GEMINI. Demais bibliotecas sao STD.

In [1]:
#Preparação do ambiente
!pip install -q -U google-generativeai

Importação das bibliotecas utilizadas

# ATENÇÂO

Para rodar é necessário fornecer sua própria chave do API

In [2]:
# Do "Get started with the Gemini API: Python"

import pathlib
import textwrap
import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Para usar a chave secreta do API guardada no COLAB,
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
#      ou
#GOOGLE_API_KEY = "COLE AQUI SUA CHAVE"

genai.configure(api_key=GOOGLE_API_KEY)

#Inicialização do modelo
model = genai.GenerativeModel('gemini-1.0-pro')

# CONFIGURAÇÂO PARA BUSCA NA INTENET

from googlesearch import search # Biblioteca não oficial para dusca no Google: https://pypi.org/project/googlesearch-python/
import requests # Biblioteca para carregar páginas da internet: https://pypi.org/project/requests/
from bs4 import BeautifulSoup # Biblioteca para manipular arquivos HTML e XML baixados da internet: https://beautiful-soup-4.readthedocs.io/en/latest/

Aqui trazemos algumas configurações importantes que podem impactar o resultado das buscas e da verificação de fatos.
A lista DOMINIOS_DE_BUSCA limita os domínios considerados confiáveis para busca de notícias e informações. Importante ter variedade para melhorar as chances de verificações cruzadas bem sucedidas. Eu incluí aqui sites de jornalismo nacionais, principais agencias do governo (todos os três poderes) e também agencias de verificação de fatos bem conceituadas, para o caso de notícias já verificadas.
Na versão atual, a ordem dos domínios não afetam os resultados da busca. Isso poderia ser modificado para priorizar agencias de checagem.

A definição da quantidade de links (NUMERO_DE_DOMINIOS) a serem buscados e analizados existe por economia de tempo e recursos. Para assuntos mais delicados e controversos, quanto mais melhor, sempre que houver uma lista grande suficiente de hits de relevantes na busca.

A definição da repetição de domínios (MAX_REPETIÇÂO) é para evitar que algum domínio domine o resultado de buscas. De pouco adianta comparar notícias de um mesmo portal, caso o portal tenha algum tipo de viés em seu editorial

In [10]:
# Definir os domínios de busca. Esta configuração é importante pois são as fontes que vão determinar
# a qualidade da informação utilizada. É importante que as fontes sejam variadas para dar um equilíbrio
# nos diferentes viéses, partindo-se do princpipio que não há jornalismo ou informação sem viés.
# Entretanto, é importante evitar sites de jornalísmo com forte viés ideológico.

NUMERO_DE_DOMINIOS = 5 # Numeros de dominios de busca para usar (quanto mais, melhor, porém mais demorado e com maior consumo de tokens)
MAX_REPETICAO = 2 # Numero máximos de repetições aceitas de links em mesmo domínio. Pouco adianta comparar 5 notícias de um mesmo portal.

# DOMÍNIOS DE BUSCA
DOMINIOS_DE_BUSCA = [
    "https://exame.com/",
    "https://www.cnnbrasil.com.br/",
    "https://www.cartacapital.com.br/",
    "https://www.poder360.com.br/",
    "https://www.bbc.com/",
    "https://g1.globo.com/",
    "https://www.poder360.com.br/", # Até aqui coloquei sites de notícias abertos
    "https://www.in.gov.br/",
    "https://www12.senado.leg.br/",
    "https://www.camara.leg.br/",
    "https://www.gov.br",
    "https://www.cnj.jus.br/", # Até aqui inclui sites oficiais mais importantes do governo federal.
    "https://agenciabrasil.ebc.com.br/",
    "https://noticias.uol.com.br/confere/",
    "https://checamos.afp.com/",
    "https://lupa.uol.com.br/",
    "https://www.boatos.org/",
    "https://projetocomprova.com.br/",
    "https://www.e-farsas.com/" # Até aqui incluí portais de verifcações de fatos profissionais, para caso a notícia já tenha sido verificada.
]



Funções utilizadas, comentadas... perdoai minha falta de fluência em programação.
Obrigado ao AI Studio por me ajudar com muitos dos trechos de código ;)


In [15]:
# Função busca de links relacionados ao critério de busca 'resultados'
def listar_links(results, dominios=DOMINIOS_DE_BUSCA, n=NUMERO_DE_DOMINIOS):
  """
  A Função executa busca no Google Search usando a variável 'resultados'
  Busca restrita aos domínios = lista de dominios
  n = numero de links a retornar, se 0 ele usa a quantidade de dominios listados
  Retorna lista de links
  """
  if n == 0:
    n = len(dominios)
  links = []
  for result in results:
    # Verificar contagem de dominio repetitivo para não ultrapassar limite estabelecido
    if verify_domains_max(result, links):
      links.append(result)
      n = n - 1
      if n == 0 :
        break
  return links

# verifica a contagens de dominios já utilizados para evitar repetições
def verify_domains_max(result, links):
  """
  retorna TRUE ou FALSE se a quantidade de dominios passou do limite
  """
  #identificar dominio
  the_domain = "new_domain"
  for domain in DOMINIOS_DE_BUSCA:
    if domain in result:
      the_domain = domain
      break
  # Contar quantas vezes dominio aparece em links
  c = 0
  for link in links:
    if the_domain in link:
      c = c + 1
  # Verifica ontagem
  if c >= MAX_REPETICAO:
    return False
  else:
    return True

# Usar Gemini para compilar textos
def compilar_noticias(links, noticias):
  """
  Le as notícias usando o Gemini e retorna JSON com titulo, conteudo e fonte
  """
  for url in links:
    print(":Status: Lendo informação:", url)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    texto_da_publicacao = soup.find('div', class_='publicacao-texto')
    try:
      # Tente executar o comando
      # Nos testes vi que muitos sites não tem título definido desta forma...
      title = soup.title.string
      print("   -> Notícia encontrada:", title)
    except Exception as error:
      # Se ocorrer um erro, execute a ação alternativa
      print(f"  -> :ERRO Menor: Noticia sem titulo - deixar titulo como 'noticia'")
      title = "Noticia"
    text_soup = soup.getText()
    try:
      gen_response = model.generate_content(
          "Transcrever o artigo principa em meio à sopa de letras e codigos:" + text_soup,
          generation_config=genai.types.GenerationConfig(
            candidate_count=1,
            temperature=0) # Temperatura zero pois a rede não deve criar, apenas transcrever
          ).text
      noticias.append({"title": title, "news": gen_response, "source": url})
      print("   -> OK para '", title, "'")
    except Exception as error:
      print("   -> ERRO: NOK para '", title, "'")
  return noticias

# Define string de busca do google search para sites de noticias
def news(fact):
  """
  Busca as notícias e retorna query para busca de referencias
  fact deve ser um link válido
  """
  # inicializa notocias
  news = []
  # Buscar notícia
  print("\n:Status: Buscando notícia fornecida...")
  response = requests.get(fact)
  soup = BeautifulSoup(response.content, 'html.parser')
  texto_da_publicacao = soup.find('div', class_='publicacao-texto')
  try:
    # Tente executar o comando
    # Nos testes vi que muitos sites não tem título definido desta forma...
    title = soup.title.string
    print("   -> Notícia encontrada:", title)
  except Exception as error:
    # Se ocorrer um erro, execute a ação alternativa
    print(f"  -> :ERRO Menor: Noticia sem titulo - deixar titulo como 'noticia'")
    title = "Noticia"
  text_soup = soup.getText()
  # Ler notícia
  print(":Status: Lendo a notícia...")
  response = model.generate_content(
      "Transcrever a seguinte sopa de letrinhas no formato do artigo principal contido nela:" + text_soup,
      generation_config=genai.types.GenerationConfig(
        candidate_count=1,
        temperature=0)
      ).text
  news.append({"title": title, "news": response, "source": fact})
  print("   -> OK")
  # Gemini cria query para buscar noticias
  print(":Status: Gerando palavras chave para busca...")
  palavras_chave = model.generate_content(
      "Criar um query com palavras-chave para buscar no google search mais notícias relacionadas a este mesmo assunto. Retorne apenas um query, sem nenhum comentário ou explicação. NOTÍCIA:" + response,
      generation_config=genai.types.GenerationConfig(
        candidate_count=1,
        temperature=0.3)
      ).text
  print("   -> Palavras-chave:", palavras_chave)
  return palavras_chave, news

#Define string de busca para fatos relatados
def fact(fact):
  """
  Cria um query com base na descrição do fato
  """
  print("\n:Status: Criando query para '" + fact +"' ...")
  palavras_chave = model.generate_content(
      "Criar um query com palavras-chave para buscar no google search notícias relacionadas a este assunto. Retorne apenas um query, sem nenhum comentário ou explicação. ASSUNTO:" + fact,
      generation_config=genai.types.GenerationConfig(
        candidate_count=1,
        temperature=0.3)
      ).text
  print("   -> Palavras-chave:", palavras_chave)
  return palavras_chave, []


# Progrma principal
def main(fact_2_check):
  if fact_2_check[0:4] == "http":
    query, noticias = news(fact_2_check)
    fact_2_check = noticias[0]['title']
  else:
    query, noticias = fact(fact_2_check)

  # Busca de noticias
  results = search(query, domains=DOMINIOS_DE_BUSCA, num=250)
  links = listar_links(results)

  # ACionamento do GEMINI para ler e compilar as notícias
  print("   -> Busca será feita em " + str(len(links)) + " domínios\n")
  noticias = compilar_noticias(links, noticias)

  # Com todas as notícias compiladas na lista notícias, fazer a comparação final
  print("\n:Status: Comparando fatos")
  prompt = "Baseado nos diferentes textos do campo 'news' e 'title' dos artigos da lista a seguir, você pode opinar sobre se fatos '" + fact_2_check + "' são ou não verdadeiros? Verifique se há consistência entre estes fatos e os damais textos 'news' fornecidos para formar sua opinião. Por favor justificar sua resposta e citar as fontes sempre que possível. Lista: " + str(noticias)
  gen_response = model.generate_content(
        prompt,
        generation_config=genai.types.GenerationConfig(
          candidate_count=1,
          temperature=1.0)
        ).text
  return gen_response

# AQUI EFETIVAMENTE RODAMOS A COISA TODA

Um "input" básico vai solicitar que se descreva um fato ou se cole um URL de notícia. Atenção que não há filtros aqui. Ele vai identificar URLs que iniciem em HTTP... se não roda como se fosse um fato descrito.

A lógica é a seguinte.

A função main() recebe o input e retorna a resolução final. Mas muita coisa acontece no meio do caminho:

Se o fato vier descrito por texto (não começa com "http") ele vai chamar a função fact() que usar o GEMINI para transformar a descrição do fato em uma query para o google, retornando esta query como texto.

Se o que entrar no input for um link (começando com "http") ele chama a função news() que baixa a notícia da internet (sem restrição de domínio, ainda) e utilisa o GEMINI para decifrar o emaranhado de html, css, scripts e tudo mais para encontrar o título e o texto da notícia. Feito isso, faz mais uma chamada ao GEMINI para transformar a notícia em uma query para busca de notícias similares no Google, retornando esta query.

À partir de agora o fluxo se unifica novamente. Com o query em mãos o programa faz uma busca no google search seguindo os critérios definidos anteriormente (domínios a verificar, número de domínios a retornar, e máximo de links por domínio). Com a lista de links em mãos o programa baixa cada um dos links e chama o GEMINI diversas vezes para identificar o texto. Assim vai criando um JSON com o título, texto e fonte de cada notícia.

Finalizada a consolidação de todas as notícias, mais uma chamada é feita, agora passando todo o JSON com a lista de notícias para que sejam comparadas.

Ao final, tem um pouco de prompt engineering para se conseguir as diversas informações, e principalmente um RAG "artesanal". Como o mecanísmo de busca do google search já tem dentro de si buscas em bases vetoriais e bases de conhecimento tipo graph (knowlegde graphs), não tivemos que nos demorar em utilizar estas técnias. A preparação dos dados para o RAG que se faz ao final foi feita com chamadas específicas, como se fossem vários agentes trabalhando juntos. Note que cada chamada tem sua engenharia de prompt e ajuste da temperatura para dar mais "eloquencia" ou exatidão ao meodelo (conforme tarefa).

Rode a vontade (com a sua chave de API) e veja que o processo é bem "verborrágico" dandos todos os detalhes das operações que vão acontecendo.

In [13]:
# ----------------------------------------------------------------------------
# AQUI EFETIVAMENTE RODAMOS O PROGRAMA
#-----------------------------------------------------------------------------
fact_2_check = input("\nCole a URL da notícia a ser verificada ou escreva sobre o fato:\n" )
result = main(fact_2_check)
to_markdown(result)



Cole a URL da notícia a ser verificada ou escreva sobre o fato:
Policia Rodoviária Federal está bloqueando caminhões com doações para o Rio Grande do Sul

:Status: Criando query para 'Policia Rodoviária Federal está bloqueando caminhões com doações para o Rio Grande do Sul' ...
   -> Palavras-chave: "Polícia Rodoviária Federal bloqueia caminhões com doações para o Rio Grande do Sul"
   -> Busca será feita em 5 domínios

:Status: Lendo informação: https://noticias.uol.com.br/confere/ultimas-noticias/2024/05/09/antt-caminhao-multa-sbt-rs.htm
   -> Notícia encontrada: O que se sabe sobre caminhões multados com doações para o RS
   -> OK para ' O que se sabe sobre caminhões multados com doações para o RS '
:Status: Lendo informação: https://lupa.uol.com.br/jornalismo/2024/05/08/o-que-se-sabe-sobre-multas-e-restricoes-a-caminhoes-com-doacoes-para-o-rs
   -> Notícia encontrada: Lupa
   -> OK para ' Lupa '
:Status: Lendo informação: https://www.gov.br/prf/pt-br/noticias/nacionais/prf-alerta-

> Com base nos textos disponíveis, **não há provas para apoiar as alegações** de que a Polícia Rodoviária Federal (PRF) esteja bloqueando caminhões com doações para o Rio Grande do Sul.
> 
> **Razões:**
> 
> * **Nenhum dos artigos mencionados relata bloqueios ou multas a caminhões transportando doações ao Rio Grande do Sul.**
> * Um artigo da UOL afirma que as multas foram aplicadas devido ao excesso de peso, não à falta de notas fiscais.
> * Outro artigo da Polícia Rodoviária Federal enfatiza a importância dos caminhões de doação e suprimentos para a região do Rio Grande do Sul.
> * Um terceiro artigo informa que a PRF está escoltando uma força-tarefa com 190 toneladas de suprimentos para as vítimas das enchentes.
> 
> Portanto, os textos fornecidos não corroboram a afirmação de que a PRF está bloqueando caminhões com doações para o Rio Grande do Sul.
> 
> **Fontes:**
> 
> * https://noticias.uol.com.br/confere/ultimas-noticias/2024/05/09/antt-caminhao-multa-sbt-rs.htm
> * https://www.gov.br/prf/pt-br/noticias/nacionais/prf-alerta-para-viagens-desnecessarias-a-porto-alegre-1
> * https://www.gov.br/prf/pt-br/noticias/estaduais/parana/2024/maio/prf-pm-e-defesa-civil-escoltam-quase-200-toneladas-de-suprimentos-para-as-vitimas-das-enchentes-no-rs

In [16]:
fact_2_check = input("\nCole a URL da notícia a ser verificada ou escreva sobre o fato:\n" )
result = main(fact_2_check)
to_markdown(result)


Cole a URL da notícia a ser verificada ou escreva sobre o fato:
https://ihu.unisinos.br/categorias/170-noticias-2014/527870-bancada-ruralista-resiste-na-definicao-de-trabalho-escravo

:Status: Buscando notícia fornecida...
   -> Notícia encontrada: Bancada ruralista resiste na definição de trabalho escravo - Instituto Humanitas Unisinos - IHU
:Status: Lendo a notícia...
   -> OK
:Status: Gerando palavras chave para busca...
   -> Palavras-chave: Trabalho escravo Brasil bancada ruralista definição
   -> Busca será feita em 5 domínios

:Status: Lendo informação: https://www.camara.leg.br/noticias/456078-COMISSAO-APROVA-PROJETO-QUE-MUDA-DEFINICAO-DE-TRABALHO-ESCRAVO-NO-CODIGO-PENAL
   -> Notícia encontrada: 
      Comissão aprova projeto que muda definição de trabalho escravo no Código Penal - Notícias - Portal da Câmara dos Deputados
  
   -> OK para ' 
      Comissão aprova projeto que muda definição de trabalho escravo no Código Penal - Notícias - Portal da Câmara dos Deputados
   '
:

> **Sim**, os fatos apresentados no artigo do Instituto Humanitas Unisinos são verdadeiros.
> 
> Os demais textos fornecidos corroboram os seguintes aspectos:
> 
> * **Resistência da bancada ruralista:** Todos os textos mencionam a resistência da bancada ruralista em alterar a definição de trabalho escravo, objetivando evitar a desapropriação de imóveis rurais por meio da Proposta de Emenda à Constituição do Trabalho Escravo (PEC do Trabalho Escravo).
> * **Mudança na definição de trabalho escravo:** O texto do Portal da Câmara dos Deputados e o de CartaCapital destacam a aprovação de um projeto de lei pela Comissão de Agricultura que altera a definição de trabalho escravo no Código Penal, excluindo os termos "jornada exaustiva" e "condições degradantes de trabalho".
> * **Dificuldade na comprovação:** O texto do Exame cita uma portaria que modifica a norma da "lista suja do trabalho escravo", tornando mais rigorosas as exigências para comprovar a prática do crime.
> * **Preocupações:** As entidades mencionadas no texto de CartaCapital expressam preocupações com as mudanças propostas, pois acreditam que elas dificultarão a caracterização de trabalho escravo em casos de terceirização.
> 
> Portanto, os fatos apresentados no artigo do IHU são consistentes com os demais textos fornecidos e apontam para uma tendência de flexibilização da definição de trabalho escravo, impulsionada pela bancada ruralista.